In [52]:
import boto3
from pprint import pprint

In [53]:
session=boto3.session.Session(profile_name='root')

In [54]:
ec2_client=session.client('ec2', 'us-east-1')

In [55]:
for each_vol in ec2_client.describe_volumes()['Volumes']:
    pprint(each_vol['VolumeId'])

'vol-08266cdbbe4ffa7dc'
'vol-08723040e790b8d88'


In [56]:
f1={'Name':'tag:prod','Values':['backup','Backup']}

list_of_volids=[]

for each_vol in ec2_client.describe_volumes(Filters=[f1])['Volumes']:
    list_of_volids.append(each_vol['VolumeId'])
    
print('The list of Volume ids is: ', list_of_volids)


The list of Volume ids is:  ['vol-08266cdbbe4ffa7dc']


In [59]:
#use paginator to get list of volumes

paginator = ec2_client.get_paginator('describe_volumes')

print(paginator.paginate(Filter=[f1]))

list_of_volidss=[]
for page in paginator.paginate(Filters=[f1]):
    for volume in page['Volumes']:
        list_of_volidss.append(volume['VolumeId'])
        
print(list_of_volidss)

['vol-08266cdbbe4ffa7dc']


In [60]:
#creating snap of each volume 

import time

snapids=[]

for volume in list_of_volidss:
    print("Taking snapshot of volume {}".format(volume))
    response= ec2_client.create_snapshot(
        Description='snapshot with lambda and cw',
        VolumeId=volume)
        
    snapids.append(response.get('SnapshotId'))
    time.sleep(15)

    
    
waiter=ec2_client.get_waiter('snapshot_completed')            
waiter.wait(SnapshotIds=snapids)

Taking snapshot of volume vol-08266cdbbe4ffa7dc
